## Imports

In [16]:
import pandas as pd
import geopandas as gpd
import os
from geopy.geocoders import GoogleV3
import geocoder
from shapely.geometry import Point
import glob

## Data read-in

In [19]:
glob.glob("*.csv")

file_list = [file for file in glob.glob('*.csv')]

In [23]:
df_list = []
for file in file_list:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [24]:
df

,PIN,category,name,address,2021 vaue,2024 value,percent change in initial assessment,2021 value,2024 value
0,17-16-130-001-0000,office,old post office,433 West Van Buren,"$854,168,112","$897,138,000",5.0%,NaN,NaN
1,17-16-120-010-0000,office,BMO Tower,310 South Canal,"$20,069,100","$532,711,000",NaN,NaN,NaN
2,17-09-326-004-0000,office,NaN,400 West Lake,"$505,992,478","$449,413,000",-11.2%,NaN,NaN
3,17-09-342-002-0000,office,Accenture Tower,500 West Madison,"$546,764,886","$412,446,000",-24.6%,NaN,NaN
4,17-09-306-033-0000,office,River Point,444 West Lake,"$432,612,009","$396,070,000",-8.4%,NaN,NaN
0,17-05-306-007-0000,industrial,NaN,1220 West Cortez,"$15,110,480","$153,010,000",913%,NaN,NaN
1,16-35-300-039-0000,industrial,NaN,3501 South Pulaski,"$17,945,328","$113,828,000",534%,NaN,NaN
2,17-09-100-004-0000,industrial,NaN,655 West Chicago,"$76,519,928","$57,987,250",-24%,NaN,NaN
3,16-24-223-001-0000,industrial,NaN,2558 West 16th,"$64,839,414","$56,178,000",-13%,NaN,NaN
4,16-16-310-011-0000,industrial,NaN,5490 West Roosevelt,"$45,751,016","$42,784,000",-6%,NaN,NaN


In [2]:
df = pd.read_csv("Top West township assessments - industrial.csv")

## Data work

In [3]:
df = df.rename(columns={'2021 vaue':'2021 value',
                       '2024 value ':'2024 value',
                       'percent change in initial assessment ':'percent change in initial assessment'})

In [4]:
df.columns

Index(['PIN', 'name', 'address', '2021 value', '2024 value',
       'percent change in initial assessment'],
      dtype='object')

In [5]:
df[['2021 value int', '2024 value int']] = df[['2021 value', '2024 value']].replace({'\$': '', ',': ''}, 
                                                                                    regex=True)

df[['2021 value int', '2024 value int']] = df[['2021 value int', '2024 value int']].astype(int)

In [6]:
df['percent change in initial assessment int'] = df['percent change in initial assessment'].replace('%',regex=True)

In [7]:
df['name'] = df['name'].fillna('Not available')

In [8]:
df['address_full'] = df['address'] + ' Chicago, IL'

In [9]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [10]:
df['geocoded'] = df['address_full'].apply(geolocator.geocode, timeout=10)

In [11]:
df['point'] = df['geocoded'].apply(lambda loc: tuple(loc.point) if loc else None)

In [12]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [13]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# Optionally, set the coordinate reference system (CRS) for the GeoDataFrame
gdf.set_crs(epsg=4326, inplace=True)  # EPSG 4326 is the standard WGS 84 latitude and longitude system

,PIN,name,address,2021 value,2024 value,percent change in initial assessment,2021 value int,2024 value int,percent change in initial assessment int,address_full,geocoded,point,lat,lon,altitude,geometry
0,17-05-306-007-0000,Not available,1220 West Cortez,"$15,110,480","$153,010,000",913%,15110480,153010000,913%,"1220 West Cortez Chicago, IL","(1220 W Cortez St, Chicago, IL 60642, USA, (41...","(41.9012462, -87.6579593, 0.0)",41.901246,-87.657959,0.0,POINT (-87.65796 41.90125)
1,16-35-300-039-0000,Not available,3501 South Pulaski,"$17,945,328","$113,828,000",534%,17945328,113828000,534%,"3501 South Pulaski Chicago, IL","(3501 S Pulaski Rd, Chicago, IL 60623, USA, (4...","(41.8308907, -87.72343219999999, 0.0)",41.830891,-87.723432,0.0,POINT (-87.72343 41.83089)
2,17-09-100-004-0000,Not available,655 West Chicago,"$76,519,928","$57,987,250",-24%,76519928,57987250,-24%,"655 West Chicago Chicago, IL","(655 W Chicago Ave, Chicago, IL 60654, USA, (4...","(41.8959156, -87.6451605, 0.0)",41.895916,-87.645161,0.0,POINT (-87.64516 41.89592)
3,16-24-223-001-0000,Not available,2558 West 16th,"$64,839,414","$56,178,000",-13%,64839414,56178000,-13%,"2558 West 16th Chicago, IL","(2558 W 16th St, Chicago, IL 60608, USA, (41.8...","(41.8602243, -87.6898621, 0.0)",41.860224,-87.689862,0.0,POINT (-87.68986 41.86022)
4,16-16-310-011-0000,Not available,5490 West Roosevelt,"$45,751,016","$42,784,000",-6%,45751016,42784000,-6%,"5490 West Roosevelt Chicago, IL","(5490 W Roosevelt Rd, Chicago, IL 60644, USA, ...","(41.8672149, -87.7617156, 0.0)",41.867215,-87.761716,0.0,POINT (-87.76172 41.86721)


In [14]:
gdf[['geocoded','point']] = gdf[['geocoded','point']].astype(str)

In [15]:
# Export to GeoJSON
gdf.to_file("top_west_assessments_06_06_24.geojson", driver='GeoJSON')